In [ ]:
from gpt import GPT
from gpt_config import GPTConfig
import torch
import pandas

In [ ]:
from torch.utils.data.dataloader import DataLoader
from sentiment_classification.reviewsDataset import reviewsDataset
from gpt_utils import dynamic_padding
import tiktoken
enc = tiktoken.get_encoding("gpt2")
def encode(s: str):
    return enc.encode(s, allowed_special={"<|endoftext|>"})


In [ ]:
# rd = reviewsDataset(split="train")
# dl = DataLoader(rd,batch_size=2,collate_fn=dynamic_padding,shuffle=True)
# for i in range(10):
    # batch = next(iter(dl))
    # print(batch["lengths"])
    # print((batch["attention_masks"]))
enc.decode([0]) 

In [ ]:
# Zero shot evaluation
from gpt import GPT
from gpt_config import GPTConfig
from gpt_utils import dynamic_padding
from torch.utils.data.dataloader import DataLoader
dl = DataLoader(rd,batch_size=2,collate_fn=dynamic_padding)
batch = next(iter(dl))
config = GPTConfig(use_lora=False)
model = GPT.from_pretrained()
model.crop_block_size(config.block_size)
print(batch["input_ids"].size())
logits, _ = model(batch["input_ids"],batch["attention_masks"])

In [ ]:
from gpt import GPT
from gpt_config import GPTConfig
import loralib as lora
import torch
model_config = GPTConfig(use_lora=True,load_from_checkpoint=True,checkpoint_path="run/finetune_lora.ckpt")
model = GPT.from_pretrained(config=model_config)
if model_config.use_lora:
    lora.mark_only_lora_as_trainable(model)
model_base = GPT.from_pretrained(config=GPTConfig(use_lora=False,block_size=128))

In [ ]:
print(dir(model_base))
param_dict_base = {pn:p for pn,p in model_base.named_parameters()}
param_dict = {pn:p for pn,p in model.named_parameters()}
for k in param_dict_base.keys():
    print(f"{k}:{torch.equal(param_dict[k],param_dict_base[k])}")

In [ ]:
# Load models
from gpt_karpathy import GPT as GPT_karpathy
from gpt import GPT
from gpt_config import GPTConfig

model_base = GPT_karpathy.from_pretrained("gpt2")
model_base.crop_block_size(128)

model = GPT.from_pretrained(config=GPTConfig(use_lora=False))

In [ ]:
from sentiment_classification.reviewsDataset import reviewsDataset
from gpt_utils import dynamic_padding
from torch.utils.data.dataloader import DataLoader
import torch
rd = reviewsDataset(split="train")
batch_rd = next(iter(rd))
dl = DataLoader(rd,batch_size=2,collate_fn=dynamic_padding)
batch = next(iter(dl))
wte_k = model_base.transformer.wte(batch_rd['input_ids'])
wte = model.transformer.wte(batch['input_ids'])
input_len = sum(batch["attention_masks"][0]).item()
# print(wte.size())
x_k = wte_k.unsqueeze(0)
for block_k in model_base.transformer.h:
    x_k = block_k(x_k)
x = wte
for block in model.transformer.h:
    x = block(x,batch["attention_masks"])
x_k = model_base.transformer.ln_f(x_k)
x_k = model_base.lm_head(x_k)
x = model.transformer.ln_f(x)
x = model.lm_head(x)

# print(torch.equal(wte_k[:input_len],wte[0][:input_len]))
# print(x[0][0:input_len],x_k.squeeze())
# print(x.size(),x_k.size())
# print(torch.isclose(x[0][0:input_len],x_k.squeeze(),atol=1e-4))
# print(batch['input_ids'][0])
# print(batch_rd["input_ids"])
# print(f"Inputs Equal: {torch.equal(batch_rd['input_ids'], batch['input_ids'])}")
logits,_ = model_base(batch_rd["input_ids"].view(1,-1))
logits_x, _ = model(batch["input_ids"],batch["attention_masks"],batch["label_idxs"])
# print(logits)
# print("---")
# print(logits_x[0])


In [ ]:
print(torch.equal(logits, logits_x))
print(logits.squeeze().size(), logits_x[0].size())
equality = torch.isclose(logits.squeeze(), logits_x[0],atol=1e-4)


Walk through a few examples of the data from encoding all the way to the attention output of the last layer

In [1]:
# Examine the data that the model is seeing.
from torch.utils.data.dataloader import DataLoader
from sentiment_classification.reviewsDataset import reviewsDataset
from gpt_utils import dynamic_padding
import tiktoken
import torch
import torch.nn.functional as F
enc = tiktoken.get_encoding("gpt2")
def encode(s: str):
    return enc.encode(s, allowed_special={"<|endoftext|>"},)

rd = reviewsDataset(split="test")
batch_size = 1
dl = DataLoader(rd,shuffle=True,batch_size=batch_size,collate_fn=dynamic_padding)

In [4]:
# Load the finetuned model
from gpt import GPT
from gpt_config import GPTConfig

model_config = GPTConfig(use_lora=False,load_from_checkpoint=True,checkpoint_path="run/dropout/finetune_no_lora.ckpt",debug=True)
# model_config = GPTConfig(debug=True,binary_classification_head=False)
model = GPT.from_pretrained(config=model_config)
ckpt = torch.load(model_config.checkpoint_path)
model.load_state_dict(ckpt["model"])
model.eval()
# device = "mps"
print("Loaded Model")


Loading pre-trained weights for gpt2
Number of parameters: 123.65M


/var/folders/p1/qmcgqwhj1mx925j0j_r36xp40000gn/T/ipykernel_75305/3948697035.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_config.checkpoint_pat

Loaded Model


In [5]:
torch.manual_seed(1336)
pos_neg = rd.get_pos_neg_indices()
for i in range(10):
    batch = next(iter(dl))
    logits, _,att_out = model(batch["input_ids"],batch["attention_masks"])
    
    predictions = F.sigmoid(logits)
    # For each attention head, pick the top 10 indices
    att_out = att_out.squeeze()
    topk_vals = []
    topk_indices = []
    for head in range(att_out.shape[0]):
        topk_v,topk_idx = torch.topk(att_out[head,:],2)
        topk_vals.append(topk_v)
        topk_indices.append(topk_idx)
    
    for j in range(batch_size):
        valid = batch["attention_masks"][j].sum()
        review = enc.decode(batch['input_ids'][j][0:valid].tolist())
        # if logits[j][pos_neg["positive"]] > logits[j][pos_neg["negative"]]:
        #     prediction = 1
        # else:
        #     prediction = 0 
        print(review)
        print(f"{batch['fpaths'][j]}")
        print(f"Label: {batch['labels'][j]}")
        print(f"Prediction: {prediction}")
        # print(f"Prediction: {predictions[j].item()}")
        words = []
        for head in range(att_out.shape[0]):
            for val,idx in zip(topk_vals[head],topk_indices[head]):
                words.append((enc.decode([batch['input_ids'][j][idx].item()]),val.item()))
                # print(f"Word: {enc.decode([batch['input_ids'][j][idx].item()])}, Attention Val: {val}")
        words = sorted(words,key=lambda x: -x[1])
        for w,c in words:
            print(f"{w},{c}")
    
    print("--------------------------------------------")

IndexError: index 33733 is out of bounds for dimension 0 with size 1